<h3>Setup</h3>

In [1]:
# Imports
import kagglehub
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [8]:
# Load Kaggle dataset
def load_kaggle_csv(dataset, filename):
    path = Path(kagglehub.dataset_download(dataset))
    return pd.read_csv(path / filename)

df = load_kaggle_csv(
    "chitwanmanchanda/fraudulent-transactions-data",
    "Fraud.csv"
)

<h3>Feature Engineering</h3>

In [9]:
# Calculate change in balance for both accounts
df['orig_delta'] = df['newbalanceOrig'] - df['oldbalanceOrg']
df['dest_delta'] = df['newbalanceDest'] - df['oldbalanceDest']

# 
df = df.drop(columns=[
    'oldbalanceOrg', 'newbalanceOrig',
    'oldbalanceDest', 'newbalanceDest',
    'isFlaggedFraud'
])

KeyError: "['oldbalanceOrg' 'newbalanceOrig' 'oldbalanceDest' 'newbalanceDest'\n 'isFlaggedFraud'] not found in axis"

In [12]:
df.head()

,step,type,amount,nameOrig,nameDest,isFraud,orig_delta,dest_delta
0,1,PAYMENT,9839.64,C1231006815,M1979787155,0,-9839.64,0.0
1,1,PAYMENT,1864.28,C1666544295,M2044282225,0,-1864.28,0.0
2,1,TRANSFER,181.00,C1305486145,C553264065,1,-181.00,0.0
3,1,CASH_OUT,181.00,C840083671,C38997010,1,-181.00,-21182.0
4,1,PAYMENT,11668.14,C2048537720,M1230701703,0,-11668.14,0.0


<h3>Machine Learning</h3>

In [3]:
X = df.drop('isFraud', axis=1)
y = df['isFraud']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [4]:
print(f"X_train shape {X_train.shape}")
print(f"X_test shape {X_test.shape}")

X_train shape (5090096, 10)
X_test shape (1272524, 10)


In [7]:
X_train

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFlaggedFraud
292779,15,PAYMENT,9914.74,C482751146,44248.00,34333.26,M1651188591,0.00,0.00,0
499763,20,PAYMENT,6854.53,C188264521,0.00,0.00,M1469015863,0.00,0.00,0
2970411,231,CASH_OUT,361211.80,C593201095,0.00,0.00,C1985763166,489745.16,850956.95,0
3137549,236,PAYMENT,7083.51,C1617277615,0.00,0.00,M1529547196,0.00,0.00,0
1500682,143,CASH_IN,218019.51,C1705563354,13045685.58,13263705.09,C2121401221,2438123.98,2220104.47,0
...,...,...,...,...,...,...,...,...,...,...
1524870,153,PAYMENT,1895.99,C1302053063,0.00,0.00,M1494049570,0.00,0.00,0
5834821,402,CASH_OUT,347110.99,C788162540,103785.00,0.00,C1771727447,87871.75,434982.74,0
4182953,304,PAYMENT,13259.63,C545341724,0.00,0.00,M253734860,0.00,0.00,0
3985280,298,PAYMENT,24122.92,C1831600144,0.00,0.00,M1382629737,0.00,0.00,0


In [5]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

ValueError: could not convert string to float: 'PAYMENT'

In [ ]:
linkcode
y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))